In [22]:
import fastbook
from fastbook import * 
from fastai.vision.all import *

In [23]:
path = untar_data(URLs.MNIST_SAMPLE)
threes = (path/'train'/'3')
sevens = (path/'train'/'7')

In [24]:
stacked_threes = torch.stack([tensor(Image.open(o)) for o in threes.ls()]).float().view(-1, 28*28)
stacked_sevens = torch.stack([tensor(Image.open(o)) for o in sevens.ls()]).float().view(-1, 28*28)
# Label below
stacked_labels = torch.cat([torch.zeros(len(stacked_threes)), torch.ones(len(stacked_sevens))])
stacked_threes.shape, stacked_sevens.shape, stacked_labels.shape

(torch.Size([6131, 784]), torch.Size([6265, 784]), torch.Size([12396]))

In [25]:
# create a dataset and dataloader for our epoch
dataset = list(zip(torch.cat([stacked_threes, stacked_sevens]), stacked_labels))
dl = DataLoader(dataset, batch_size=64, shuffle=True) 

In [26]:
def batch_accuracy(preds, targs):
    preds = preds.sigmoid()
    return (preds > 0.5).eq(targs).float().mean()

In [41]:
def validate_epoch(model):
    accs = [batch_accuracy(model(x),y) for x,y in dl]
    return round(torch.stack(accs).mean().item(),4)

In [28]:
linear_model = nn.Linear(28*28, 1)

In [29]:
w,b = linear_model.parameters()
w.shape, b.shape

(torch.Size([1, 784]), torch.Size([1]))

In [30]:
class BasicOptimizer:
    def __init__(self, params, lr=0.01):
        self.params = list(params)
        self.lr = lr

    def step(self):
        for p in self.params:
            p.data -= self.lr * p.grad.data

    def zero_grad(self):
        for p in self.params:
            p.grad = None

In [31]:
opt = BasicOptimizer(linear_model.parameters(), lr=0.01)

In [32]:
def mnist_loss(preds, targs):
    preds_sigmoid = preds.sigmoid()
    return torch.where(targs == 1, 1-preds_sigmoid,  preds_sigmoid).mean()

In [33]:
def calc_grad(x, y, model):
    preds = model(x)
    loss = mnist_loss(preds, y)
    loss.backward()
    return loss

In [34]:
# Start the epoch for the calculation
def train_epoch(dl, model, opt):
    for x, y in dl:
        calc_grad(x, y, model)
        opt.step()
        opt.zero_grad()

        

In [35]:
def train_model(model,  epochs=1):
    for epoch in range(epochs):
        train_epoch(dl, model, opt)
        acc = validate_epoch(model)
        print(f'Epoch {epoch+1}/{epochs}, Accuracy: {acc}')

In [44]:
train_model(linear_model, epochs=10)

Epoch 1/10, Accuracy: 0.5073
Epoch 2/10, Accuracy: 0.5072
Epoch 3/10, Accuracy: 0.5085
Epoch 4/10, Accuracy: 0.5078
Epoch 5/10, Accuracy: 0.5082
Epoch 6/10, Accuracy: 0.5065
Epoch 7/10, Accuracy: 0.5073
Epoch 8/10, Accuracy: 0.5068
Epoch 9/10, Accuracy: 0.5074
Epoch 10/10, Accuracy: 0.5085
